In [1]:
import sys
sys.path.append('../scripts/')
from puddle_world import *
import itertools 

In [2]:
class PolicyEvaluator:   ###policyevaluator4
    def __init__(self, widths, goal, time_interval, lowerleft=np.array([-4, -4]).T, upperright=np.array([4, 4]).T): #time_intervalを追加
        self.pose_min = np.r_[lowerleft, 0]
        self.pose_max = np.r_[upperright, math.pi*2]
        self.widths = widths
        self.goal = goal
        
        self.index_nums = ((self.pose_max - self.pose_min)/self.widths).astype(int)
        nx, ny, nt = self.index_nums
        self.indexes = list(itertools.product(range(nx), range(ny), range(nt)))
        
        self.value_function =  self.init_value_function()
        self.policy = self.init_policy()
        self.actions = set([tuple(self.policy[i]) for i in self.indexes]) 
        
        self.state_transition_probs = self.init_state_transition_probs(time_interval) #追加
        
    def init_policy(self):
        tmp = np.zeros(np.r_[self.index_nums,2]) #制御出力が2次元なので、配列の次元を4次元に
        for index in self.indexes:
            center = self.pose_min + self.widths*(np.array(index).T + 0.5)  #セルの中心の座標
            tmp[index] = PuddleIgnoreAgent.policy(center, self.goal)
            
        return tmp
    
    def init_state_transition_probs(self, time_interval): #追加
        dx = np.linspace(0, self.widths[0], 5)
        dy = np.linspace(0, self.widths[1], 5)
        dt = np.linspace(0, self.widths[2], 5)
        samples = list(itertools.product(dx, dy, dt))
        
        tmp = {}
        for a in self.actions:
            for k in range(self.index_nums[2]):
                for s in samples:
                    before = np.array([s[0], s[1], s[2] + k*self.widths[2]]).T
                    after = IdealRobot.state_transition(a[0], a[1], time_interval, before)
                    
                    index = np.floor((after - self.pose_min)/self.widths).astype(int)
                    index[2] = (index[2] + self.index_nums[2])%self.index_nums[2] #方角のインデックスを正規化
                    print(index)
                transitions = []
                
        return tmp
        
    def init_value_function(self):
        tmp = np.empty(self.index_nums) #全離散状態を要素に持つ配列を作成
        
        for index in self.indexes: 
            tmp[index] = self.goal.value if self.final_state(np.array(index).T) else 1e+10
            
        return tmp
        
    def final_state(self, index):
        x_min, y_min, _ = self.pose_min + self.widths*index          #xy平面で左下の座標
        x_max, y_max, _ = self.pose_min + self.widths*(index + 1) #右上の座標（斜め上の離散状態の左下の座標）
        
        corners = [[x_min, y_min, _], [x_min, y_max, _], [x_max, y_min, _], [x_max, y_max, _] ] #4隅の座標
        return sum([self.goal.inside(np.array(c).T) for c in corners ]) == 4 #全部goal.insideがTrueならsumが4に
        
    def pose_to_index(self, pose):
        while pose[2] < 0.0:            pose[2] += math.pi*2
        while pose[2] >= math.pi*2: pose[2] -= math.pi*2
        
        indexes = np.floor((pose - self.pose_min)/self.widths).astype(int)
        if not 0 <= indexes[0] < self.index_nums[0]: return None
        if not 0 <= indexes[1] < self.index_nums[1]: return None
        
        return indexes

In [3]:
pe = PolicyEvaluator(np.array([0.2, 0.2, math.pi/18]).T, Goal(-3,-3), 0.1)  

[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 21  0]
[20 21  0]
[20 21  0]
[20 21  0]
[20 21  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 21  0]
[20 21  0]
[20 21  0]
[20 21  0]
[20 21  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 21  0]
[20 21  0]
[20 21  0]
[20 21  0]
[20 21  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  0]
[20 20  1]
[20 20  0]

[20 20  7]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  8]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  8]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  8]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  8]
[20 21  7]
[20 21  7]
[20 21  7]
[20 21  7]
[20 21  8]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  8]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  8]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  8]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  8]
[20 21  7]
[20 21  7]
[20 21  7]
[20 21  7]
[20 21  8]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  8]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  8]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  8]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  8]
[20 21  7]
[20 21  7]
[20 21  7]
[20 21  7]
[20 21  8]
[21 20  7]
[21 20  7]
[21 20  7]
[21 20  7]
[21 20  8]
[21 20  7]
[21 20  7]
[21 20  7]
[21 20  7]
[21 20  8]
[21 20  7]
[21 20  7]
[21 20  7]
[21 20  7]
[21 20  8]
[21 20  7]

[20 21 15]
[20 21 16]
[19 20 16]
[19 20 16]
[19 20 16]
[19 20 16]
[19 20 17]
[19 20 16]
[19 20 16]
[19 20 16]
[19 20 16]
[19 20 17]
[19 20 16]
[19 20 16]
[19 20 16]
[19 20 16]
[19 20 17]
[19 20 16]
[19 20 16]
[19 20 16]
[19 20 16]
[19 20 17]
[19 21 16]
[19 21 16]
[19 21 16]
[19 21 16]
[19 21 17]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 17]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 17]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 17]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 17]
[20 21 16]
[20 21 16]
[20 21 16]
[20 21 16]
[20 21 17]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 17]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 17]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 17]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 17]
[20 21 16]
[20 21 16]
[20 21 16]
[20 21 16]
[20 21 17]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 17]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 17]
[20 20 16]
[20 20 16]
[20 20 16]
[20 20 16]

[20 20 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 19 28]
[20 19 28]
[20 19 28]
[20 19 28]
[20 19 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 19 28]
[20 19 28]
[20 19 28]
[20 19 28]
[20 19 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 19 28]
[20 19 28]
[20 19 28]
[20 19 28]
[20 19 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 28]
[20 20 29]

[20 20  4]
[20 20  5]
[20 20  5]
[20 20  4]
[20 20  4]
[20 20  4]
[20 20  5]
[20 20  5]
[20 20  4]
[20 20  4]
[20 20  4]
[20 20  5]
[20 20  5]
[20 20  4]
[20 20  4]
[20 20  4]
[20 20  5]
[20 20  5]
[20 21  4]
[20 21  4]
[20 21  4]
[20 21  5]
[20 21  5]
[21 20  4]
[21 20  4]
[21 20  4]
[21 20  5]
[21 20  5]
[21 20  4]
[21 20  4]
[21 20  4]
[21 20  5]
[21 20  5]
[21 20  4]
[21 20  4]
[21 20  4]
[21 20  5]
[21 20  5]
[21 20  4]
[21 20  4]
[21 20  4]
[21 20  5]
[21 20  5]
[21 21  4]
[21 21  4]
[21 21  4]
[21 21  5]
[21 21  5]
[20 20  5]
[20 20  5]
[20 20  5]
[20 20  6]
[20 20  6]
[20 20  5]
[20 20  5]
[20 20  5]
[20 20  6]
[20 20  6]
[20 20  5]
[20 20  5]
[20 20  5]
[20 20  6]
[20 20  6]
[20 20  5]
[20 20  5]
[20 20  5]
[20 20  6]
[20 20  6]
[20 21  5]
[20 21  5]
[20 21  5]
[20 21  6]
[20 21  6]
[20 20  5]
[20 20  5]
[20 20  5]
[20 20  6]
[20 20  6]
[20 20  5]
[20 20  5]
[20 20  5]
[20 20  6]
[20 20  6]
[20 20  5]
[20 20  5]
[20 20  5]
[20 20  6]
[20 20  6]
[20 20  5]
[20 20  5]
[20 20  5]

[21 20 16]
[21 20 16]
[21 20 17]
[21 20 17]
[21 20 16]
[21 20 16]
[21 20 16]
[21 20 17]
[21 20 17]
[21 21 16]
[21 21 16]
[21 21 16]
[21 21 17]
[21 21 17]
[20 20 17]
[20 20 17]
[20 20 17]
[20 20 18]
[20 20 18]
[20 20 17]
[20 20 17]
[20 20 17]
[20 20 18]
[20 20 18]
[20 20 17]
[20 20 17]
[20 20 17]
[20 20 18]
[20 20 18]
[20 20 17]
[20 20 17]
[20 20 17]
[20 20 18]
[20 20 18]
[20 21 17]
[20 21 17]
[20 21 17]
[20 21 18]
[20 21 18]
[20 20 17]
[20 20 17]
[20 20 17]
[20 20 18]
[20 20 18]
[20 20 17]
[20 20 17]
[20 20 17]
[20 20 18]
[20 20 18]
[20 20 17]
[20 20 17]
[20 20 17]
[20 20 18]
[20 20 18]
[20 20 17]
[20 20 17]
[20 20 17]
[20 20 18]
[20 20 18]
[20 21 17]
[20 21 17]
[20 21 17]
[20 21 18]
[20 21 18]
[20 20 17]
[20 20 17]
[20 20 17]
[20 20 18]
[20 20 18]
[20 20 17]
[20 20 17]
[20 20 17]
[20 20 18]
[20 20 18]
[20 20 17]
[20 20 17]
[20 20 17]
[20 20 18]
[20 20 18]
[20 20 17]
[20 20 17]
[20 20 17]
[20 20 18]
[20 20 18]
[20 21 17]
[20 21 17]
[20 21 17]
[20 21 18]
[20 21 18]
[20 20 17]
[20 20 17]

[20 20 29]
[20 20 29]
[20 20 29]
[20 20 30]
[20 20 30]
[20 20 29]
[20 20 29]
[20 20 29]
[20 20 30]
[20 20 30]
[20 20 29]
[20 20 29]
[20 20 29]
[20 20 30]
[20 20 30]
[20 21 29]
[20 21 29]
[20 21 29]
[20 21 30]
[20 21 30]
[20 20 29]
[20 20 29]
[20 20 29]
[20 20 30]
[20 20 30]
[20 20 29]
[20 20 29]
[20 20 29]
[20 20 30]
[20 20 30]
[20 20 29]
[20 20 29]
[20 20 29]
[20 20 30]
[20 20 30]
[20 20 29]
[20 20 29]
[20 20 29]
[20 20 30]
[20 20 30]
[20 21 29]
[20 21 29]
[20 21 29]
[20 21 30]
[20 21 30]
[21 20 29]
[21 20 29]
[21 20 29]
[21 20 30]
[21 20 30]
[21 20 29]
[21 20 29]
[21 20 29]
[21 20 30]
[21 20 30]
[21 20 29]
[21 20 29]
[21 20 29]
[21 20 30]
[21 20 30]
[21 20 29]
[21 20 29]
[21 20 29]
[21 20 30]
[21 20 30]
[21 21 29]
[21 21 29]
[21 21 29]
[21 21 30]
[21 21 30]
[20 20 30]
[20 20 30]
[20 20 30]
[20 20 31]
[20 20 31]
[20 20 30]
[20 20 30]
[20 20 30]
[20 20 31]
[20 20 31]
[20 20 30]
[20 20 30]
[20 20 30]
[20 20 31]
[20 20 31]
[20 20 30]
[20 20 30]
[20 20 30]
[20 20 31]
[20 20 31]
[20 21 30]

[20 20  6]
[20 21  5]
[20 21  5]
[20 21  6]
[20 21  6]
[20 21  6]
[21 20  5]
[21 20  5]
[21 20  6]
[21 20  6]
[21 20  6]
[21 20  5]
[21 20  5]
[21 20  6]
[21 20  6]
[21 20  6]
[21 20  5]
[21 20  5]
[21 20  6]
[21 20  6]
[21 20  6]
[21 20  5]
[21 20  5]
[21 20  6]
[21 20  6]
[21 20  6]
[21 21  5]
[21 21  5]
[21 21  6]
[21 21  6]
[21 21  6]
[20 20  6]
[20 20  6]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  6]
[20 20  6]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  6]
[20 20  6]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  6]
[20 20  6]
[20 20  7]
[20 20  7]
[20 20  7]
[20 21  6]
[20 21  6]
[20 21  7]
[20 21  7]
[20 21  7]
[20 20  6]
[20 20  6]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  6]
[20 20  6]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  6]
[20 20  6]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  6]
[20 20  6]
[20 20  7]
[20 20  7]
[20 20  7]
[20 21  6]
[20 21  6]
[20 21  7]
[20 21  7]
[20 21  7]
[20 20  6]
[20 20  6]
[20 20  7]
[20 20  7]
[20 20  7]
[20 20  6]
[20 20  6]
[20 20  7]
[20 20  7]
[20 20  7]

[20 20 19]
[20 20 19]
[20 20 20]
[20 20 20]
[20 20 20]
[20 20 19]
[20 20 19]
[20 20 20]
[20 20 20]
[20 20 20]
[20 20 19]
[20 20 19]
[20 20 20]
[20 20 20]
[20 20 20]
[20 20 19]
[20 20 19]
[20 20 20]
[20 20 20]
[20 20 20]
[20 21 19]
[20 21 19]
[20 21 20]
[20 21 20]
[20 21 20]
[20 20 19]
[20 20 19]
[20 20 20]
[20 20 20]
[20 20 20]
[20 20 19]
[20 20 19]
[20 20 20]
[20 20 20]
[20 20 20]
[20 20 19]
[20 20 19]
[20 20 20]
[20 20 20]
[20 20 20]
[20 20 19]
[20 20 19]
[20 20 20]
[20 20 20]
[20 20 20]
[20 21 19]
[20 21 19]
[20 21 20]
[20 21 20]
[20 21 20]
[20 20 19]
[20 20 19]
[20 20 20]
[20 20 20]
[20 20 20]
[20 20 19]
[20 20 19]
[20 20 20]
[20 20 20]
[20 20 20]
[20 20 19]
[20 20 19]
[20 20 20]
[20 20 20]
[20 20 20]
[20 20 19]
[20 20 19]
[20 20 20]
[20 20 20]
[20 20 20]
[20 21 19]
[20 21 19]
[20 21 20]
[20 21 20]
[20 21 20]
[21 20 19]
[21 20 19]
[21 20 20]
[21 20 20]
[21 20 20]
[21 20 19]
[21 20 19]
[21 20 20]
[21 20 20]
[21 20 20]
[21 20 19]
[21 20 19]
[21 20 20]
[21 20 20]
[21 20 20]
[21 20 19]

[20 20 35]
[20 20 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 21 34]
[20 21 34]
[20 21 35]
[20 21 35]
[20 21 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 21 34]
[20 21 34]
[20 21 35]
[20 21 35]
[20 21 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 21 34]
[20 21 34]
[20 21 35]
[20 21 35]
[20 21 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 20 34]
[20 20 34]
[20 20 35]
[20 20 35]
[20 20 35]
[20 21 34]
[20 21 34]
[20 21 35]
[20 21 35]

In [4]:
np.arange(0, 5.1, 5/10)

array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ])

In [5]:
np.linspace(0,1,10)

array([ 0.        ,  0.11111111,  0.22222222,  0.33333333,  0.44444444,
        0.55555556,  0.66666667,  0.77777778,  0.88888889,  1.        ])